# Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from itertools import permutations
import requests
from tqdm import tqdm
import os
import pickle
import glob

# Explore all datasets

## METR-LA

In [ ]:
df = pd.read_hdf('data/METRLA/metr-la.h5')
df.head()

In [ ]:
df.tail()

In [ ]:
df.describe()

In [ ]:
total_entries = df.size
zero_entries = (df == 0).sum().sum()
percentage_zeros = (zero_entries / total_entries) * 100
percentage_zeros

## PEMS-BAY

In [ ]:
df2 = pd.read_hdf('data/PEMSBAY/PEMS-BAY.h5')
df2.head()

In [ ]:
df2.tail()

In [ ]:
df2.describe()

In [ ]:
total_entries = df2.size
zero_entries = (df2 == 0).sum().sum()
percentage_zeros = (zero_entries / total_entries) * 100
percentage_zeros

## HAUGE

In [ ]:
df3 = pd.read_hdf('data/Hauge/hague_comp_filled.h5', sep=';')

In [ ]:
df3.head()

In [ ]:
df3.tail()

In [ ]:
df3.describe()

In [ ]:
total_entries = df3.size
zero_entries = (df3 == 0).sum().sum()
percentage_zeros = (zero_entries / total_entries) * 100
percentage_zeros

In [ ]:
df4 = pd.read_hdf('data/Hauge/hague_filled.h5', sep=';')

In [ ]:
df4.describe()

Original data for one specific intersection. Legacy code below

In [ ]:
df3 = df3.iloc[:-1]
df3['K074'] = pd.to_datetime(df3['K074'])
df3_first_day = df3[df3['K074'].dt.date == df3['K074'].dt.date.min()]

# Plot the distribution of values in column '021' over time
plt.figure(figsize=(10, 6))
plt.plot(df3_first_day['K074'], df3_first_day['021'], marker='o')
plt.title('Distribution of Sensor 021 at Area K074 on 01/01/2012')
plt.xlabel('Time')
plt.ylabel('021 Values')
plt.xticks(rotation=45)
plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels
plt.show()

In [ ]:
df4 = pd.read_csv('data/Hauge/K415/K415-2018-1-.csv', sep=';')
df4 = df4.drop(df4.index[-1])
df4.columns

In [ ]:
df4.describe()

## Check adjacency matrix
### METR-LA

In [ ]:
import pickle

file_path = 'data/METRLA/adj_mx.pkl'

with open(file_path, 'rb') as f:
    adj_mx = pickle.load(f, encoding='latin1')

print(len(adj_mx))
print(adj_mx[0])
print(len(adj_mx[0]))
print(adj_mx[1])


In [ ]:
len(adj_mx[2][0])

In [ ]:
adj_mx[2]

# Calculate new distance

## Test on two points

In [ ]:
# osrm_server = "http://router.project-osrm.org"
osrm_server = "http://127.0.0.1:5001"
x1 = '4.229222222222222'
y1 = '52.05197222222222'

x2 = '4.228972222222223'
y2 = '52.05175'


request_url = f"{osrm_server}/route/v1/driving/{x1},{y1};{x2},{y2}?overview=false"
print(request_url)
# Making the request to the OSRM API
response = requests.get(request_url)
route_data = response.json()
print(route_data)

# Parsing the distance from the response
# Note: Make sure to handle any errors or unexpected response formats in a real application
if route_data['code'] == 'Ok':
    cost = route_data['routes'][0]['distance']  # Distance in meters
    print(f"The distance between the two points is {cost} meters")

### Illustration of two points on map

In [ ]:
import requests
from geopy.distance import geodesic
import folium
from IPython.display import display

def compute_and_visualize_distances(sensor_a, sensor_b):
    osrm_server = "http://router.project-osrm.org"

    # Compute the direct distance using the Haversine formula
    direct_distance = geodesic((sensor_a['latitude'], sensor_a['longitude']), (sensor_b['latitude'], sensor_b['longitude'])).meters
    
    # Compute the routing distance using the OSRM API
    request_url = f"{osrm_server}/route/v1/driving/{sensor_a['longitude']},{sensor_a['latitude']};{sensor_b['longitude']},{sensor_b['latitude']}?overview=full&geometries=geojson"
    response = requests.get(request_url)
    route_data = response.json()
    
    if route_data['code'] == 'Ok':
        routing_distance = route_data['routes'][0]['distance']  # Distance in meters
        route_geometry = route_data['routes'][0]['geometry']
    else:
        routing_distance = float('inf')  # An arbitrary large number to denote failure to get distance
        route_geometry = None
    
    # Create a map centered around the midpoint of the two sensors
    midpoint = ((sensor_a['latitude'] + sensor_b['latitude']) / 2, (sensor_a['longitude'] + sensor_b['longitude']) / 2)
    m = folium.Map(location=midpoint, zoom_start=13, tiles='CartoDB Positron')
    
    # Add markers for the two sensors
    folium.Marker([sensor_a['latitude'], sensor_a['longitude']], popup=f"Sensor A: {sensor_a['sensor_id']}").add_to(m)
    folium.Marker([sensor_b['latitude'], sensor_b['longitude']], popup=f"Sensor B: {sensor_b['sensor_id']}").add_to(m)
    
    # Draw a line for the direct distance
    folium.PolyLine(
        locations=[(sensor_a['latitude'], sensor_a['longitude']), (sensor_b['latitude'], sensor_b['longitude'])],
        color='blue',
        weight=2,
        opacity=0.6,
        tooltip=f"Direct distance: {direct_distance:.2f} meters"
    ).add_to(m)
    
    # Draw a line for the routing distance if available
    if route_geometry:
        folium.PolyLine(
            locations=[(coord[1], coord[0]) for coord in route_geometry['coordinates']],
            color='green',
            weight=2,
            opacity=0.6,
            tooltip=f"Routing distance: {routing_distance:.2f} meters"
        ).add_to(m)
    
    # Display the map
    display(m)

    # Print distances
    print(f"Direct distance: {direct_distance:.2f} meters")
    print(f"Routing distance: {routing_distance:.2f} meters")


In [ ]:
sensor_a = {'sensor_id': 9, 'latitude': 34.15562, 'longitude': -118.46860}  # Example coordinates for sensor A
sensor_b = {'sensor_id': 75, 'latitude': 34.15571, 'longitude': -118.43273}  # Example coordinates for sensor B

compute_and_visualize_distances(sensor_a, sensor_b)

## Hague

In [ ]:
def read_sensors_and_group_by_trajectory(csv_file_path):
    trajectory_groups = {'T1N': [], 'T1S': [], 'T2N': [], 'T2S': []}
    with open(csv_file_path, mode='r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            trajectories = row['trajectory'].split('+')
            for trajectory in trajectories:
                trajectory_groups[trajectory].append(row)
    return trajectory_groups

def calculate_distances_osrm_grouped(sensors_grouped):
    # osrm_server = "http://router.project-osrm.org"
    osrm_server = "http://127.0.0.1:5001"
    results = []

    for trajectory, sensors in sensors_grouped.items():
        # Prepare the combinations and wrap it with tqdm for the progress bar
        sensor_pairs = list(permutations(sensors, 2))
        for sensor_a, sensor_b in tqdm(sensor_pairs, desc=f"Calculating distances for {trajectory}"):
            request_url = f"{osrm_server}/route/v1/driving/{sensor_a['longitude']},{sensor_a['latitude']};{sensor_b['longitude']},{sensor_b['latitude']}?overview=false"
            
            response = requests.get(request_url)
            route_data = response.json()
            
            if route_data['code'] == 'Ok':
                cost = route_data['routes'][0]['distance']  # Distance in meters
            else:
                cost = float('inf')  # Indicate failure
            
            results.append({
                'trajectory': trajectory,
                'from': sensor_a['sensor_id'],
                'to': sensor_b['sensor_id'],
                'cost': cost
            })
            # print(f"From {sensor_a['sensor_id']} to {sensor_b['sensor_id']} cost: {cost}")

    return results


In [ ]:
csv_file_path = 'data/Hauge/location_std.csv'
sensors_grouped = read_sensors_and_group_by_trajectory(csv_file_path)
results = calculate_distances_osrm_grouped(sensors_grouped)


Save distances betwee each sensor on the same trajectory into a csv file

In [ ]:
with open('data/Hauge/distances.csv', mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['trajectory', 'from', 'to', 'cost'])
    writer.writeheader()
    writer.writerows(results)

## METR-LA

In [ ]:
def read_sensors(csv_file_path):
    sensors = []
    with open(csv_file_path, mode='r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            sensors.append(row)
    return sensors

In [ ]:
def calculate_distances_osrm(sensors):
    results = []
    osrm_server = "http://router.project-osrm.org"  # Example server, replace with your own if you have one
    # osrm_server = "http://127.0.0.1:5001"
    for sensor_a in tqdm(sensors, desc="Calculating distances"):
        for sensor_b in sensors:
            if sensor_a['sensor_id'] == sensor_b['sensor_id']:
                cost = 0.0  # Cost is zero when it's the same sensor
            else:
                # Constructing the request URL
                request_url = f"{osrm_server}/route/v1/driving/{sensor_a['longitude']},{sensor_a['latitude']};{sensor_b['longitude']},{sensor_b['latitude']}?overview=false"
                print(request_url)
                # Making the request to the OSRM API
                response = requests.get(request_url)
                route_data = response.json()
                
                # Parsing the distance from the response
                # Note: Make sure to handle any errors or unexpected response formats in a real application
                if route_data['code'] == 'Ok':
                    cost = route_data['routes'][0]['distance']  # Distance in meters
                else:
                    cost = float('inf')  # An arbitrary large number to denote failure to get distance
                # print(cost)
            results.append({'from': sensor_a['sensor_id'], 'to': sensor_b['sensor_id'], 'cost': cost})
            # print(results)
    return results

In [ ]:
csv_file_path = 'data/METRLA/graph_sensor_locations.csv'
output_csv_file_path = 'data/METRLA/distances_la_route.csv'

sensors = read_sensors(csv_file_path)
distance_results = calculate_distances_osrm(sensors)

In [ ]:
with open(output_csv_file_path, mode='w', newline='') as csvfile:
    fieldnames = ['from', 'to', 'cost']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for result in distance_results:
        writer.writerow(result)

print(f"Distances saved to {output_csv_file_path}")

## PeMS-BAY

In [ ]:
csv_file_path = 'data/PEMSBAY/graph_sensor_locations_bay.csv'
output_csv_file_path = 'data/PEMSBAY/distances_bay_route.csv'
sensors = read_sensors(csv_file_path)
distance_results = calculate_distances_osrm(sensors)

In [ ]:
with open(output_csv_file_path, mode='w', newline='') as csvfile:
    fieldnames = ['from', 'to', 'cost']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for result in distance_results:
        writer.writerow(result)

print(f"Distances saved to {output_csv_file_path}")

# Create new adjacency matrix

## METR-LA

In [ ]:
distances_df = pd.read_csv('data/METRLA/distances_la_route.csv')
df = pd.read_hdf('data/METRLA/metr-la.h5')
df.head()

In [ ]:
def get_adjacency_matrix(dist_mx_array, normalized_k=0.1):
    # Calculate the standard deviation as theta for normalization
    distances = dist_mx_array[~np.isinf(dist_mx_array)].flatten()
    std = distances.std()
    adj_mx_array = np.exp(-np.square(dist_mx_array / std))
    
    # Set entries below a threshold to zero for sparsity
    adj_mx_array[adj_mx_array < normalized_k] = 0
    return adj_mx_array

In [ ]:
sensor_ids = df.columns.astype(str).tolist()

# Efficiently convert sensor IDs to index for quick lookup (now all as strings)
sensor_id_to_index = {str(sensor_id): index for index, sensor_id in enumerate(sensor_ids)}

# Initialize an empty distance matrix
num_sensors = len(sensor_ids)
dist_mx_array = np.full((num_sensors, num_sensors), np.inf)

# Make sure 'from' and 'to' in distances_df are also strings
distances_df['from'] = distances_df['from'].astype(str)
distances_df['to'] = distances_df['to'].astype(str)


In [ ]:
# Populate the distance matrix with the data from distances_df
for _, row in distances_df.iterrows():
    from_sensor = row['from']
    to_sensor = row['to']
    if from_sensor in sensor_id_to_index and to_sensor in sensor_id_to_index:
        i = sensor_id_to_index[from_sensor]
        j = sensor_id_to_index[to_sensor]
        dist_mx_array[i, j] = row['cost']

# Replace the diagonal with zeros since the distance from a sensor to itself is zero
np.fill_diagonal(dist_mx_array, 0)

In [ ]:
adj_mx_array = get_adjacency_matrix(dist_mx_array).astype(np.float32)
adj_mx = [sensor_ids, sensor_id_to_index, adj_mx_array]

In [ ]:
adj_mx[2]

In [ ]:
directory = 'data/METRLA'
file_name = 'adj_mx_new.pkl'
file_path = os.path.join(directory, file_name)

# Ensure the directory exists
os.makedirs(directory, exist_ok=True)

# Save the adj_mx to a .pkl file
with open(file_path, 'wb') as f:
    pickle.dump(adj_mx, f)

## PeMS-BAY

In [ ]:
# Load distances from CSV
distances_df = pd.read_csv('data/PEMSBAY/distances_bay_route.csv')
df = pd.read_hdf('data/PEMSBAY/pems-bay.h5')

In [ ]:
def get_adjacency_matrix(dist_mx_array, normalized_k=0.1):
    # Calculate the standard deviation as theta for normalization
    distances = dist_mx_array[~np.isinf(dist_mx_array)].flatten()
    std = distances.std()
    adj_mx_array = np.exp(-np.square(dist_mx_array / std))
    
    # Set entries below a threshold to zero for sparsity
    adj_mx_array[adj_mx_array < normalized_k] = 0
    return adj_mx_array

In [ ]:
sensor_ids = df.columns.astype(str).tolist()

# Efficiently convert sensor IDs to index for quick lookup (now all as strings)
sensor_id_to_index = {str(sensor_id): index for index, sensor_id in enumerate(sensor_ids)}

# Initialize an empty distance matrix
num_sensors = len(sensor_ids)
dist_mx_array = np.full((num_sensors, num_sensors), np.inf)

# Make sure 'from' and 'to' in distances_df are also strings
distances_df['from'] = distances_df['from'].astype(str)
distances_df['to'] = distances_df['to'].astype(str)


In [ ]:
# Populate the distance matrix with the data from distances_df
for _, row in distances_df.iterrows():
    from_sensor = row['from']
    to_sensor = row['to']
    if from_sensor in sensor_id_to_index and to_sensor in sensor_id_to_index:
        i = sensor_id_to_index[from_sensor]
        j = sensor_id_to_index[to_sensor]
        dist_mx_array[i, j] = row['cost']

# Replace the diagonal with zeros since the distance from a sensor to itself is zero
np.fill_diagonal(dist_mx_array, 0)

In [ ]:
adj_mx_array = get_adjacency_matrix(dist_mx_array).astype(np.float32)

# Construct the adj_mx list as specified with the updated adj_mx_array
adj_mx = [sensor_ids, sensor_id_to_index, adj_mx_array]

In [ ]:
adj_mx[2]

In [ ]:
directory = 'data/PEMSBAY'
file_name = 'adj_mx_new.pkl'
file_path = os.path.join(directory, file_name)

# Ensure the directory exists
os.makedirs(directory, exist_ok=True)

# Save the adj_mx to a .pkl file
with open(file_path, 'wb') as f:
    pickle.dump(adj_mx, f)

# Cleansing Hague data

K159-2018-7-1

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("data/Hauge/K159/K159-2018-7-1.csv", delimiter=';')

# Convert the first column to datetime, assuming the format "day-month-year hours:minutes"
# Note: Adjust the 'dayfirst=True' parameter if your date format varies
first_column_name = df.columns[0]
df[first_column_name] = pd.to_datetime(df[first_column_name], dayfirst=True)

# Format the date in the desired output format "year-month-day hours:minutes"
df[first_column_name] = df[first_column_name].dt.strftime('%Y-%m-%d %H:%M')

In [ ]:
df.to_csv("K159-2018-7-1_modified.csv", sep=';', index=False)

K159-2018-7-2

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("data/Hauge/K159/K159-2018-7-2.csv", delimiter=';')
df.loc[:len(df)-2, first_column_name] = pd.to_datetime(df[first_column_name].iloc[:-1], dayfirst=True).dt.strftime('%Y-%m-%d %H:%M')

In [ ]:
df.to_csv("K159-2018-7-2_modified.csv", sep=';', index=False)

K182-2019-9-1

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("data/Hauge/K182/K182-2019-9-1.csv", delimiter=';')

# Convert the first column to datetime, assuming the format "day-month-year hours:minutes"
# Note: Adjust the 'dayfirst=True' parameter if your date format varies
first_column_name = df.columns[0]
df[first_column_name] = pd.to_datetime(df[first_column_name], dayfirst=True)

# Format the date in the desired output format "year-month-day hours:minutes"
df[first_column_name] = df[first_column_name].dt.strftime('%Y-%m-%d %H:%M')

In [ ]:
df.to_csv("K182-2019-9-1_modified.csv", sep=';', index=False)

K182-2019-9-2

In [ ]:
df = pd.read_csv("data/Hauge/K182/K182-2019-9-2.csv", delimiter=';')
df.loc[:len(df)-2, first_column_name] = pd.to_datetime(df[first_column_name].iloc[:-1], dayfirst=True).dt.strftime('%Y-%m-%d %H:%M')

In [ ]:
df.to_csv("K182-2019-9-2_modified.csv", sep=';', index=False)

# Create adjacency matrix HAUGE

In [ ]:
sensors_dict = {
        'K502': ['082', '051', '111', '081'],
        'K504': ['08_2', '12_1', '02_1', '08_1', '11_1', '09_1', '12_2', '02_2', '04_2', '05_1', '04_1', '05_2'],
        'K503': ['02_1', '03_1', '11_1', '07_1', '09_1', '05_1', '05_2', '11_2'],
        'K263': ['052', '041', '061', '112', '051', '031', '111', '081'],
        'K556': ['08_2', '02_1', '08_1', '11_1', '02_2', '10_1', '05_1', '04_1'],
        'K557': ['08_2', '02_1', '08_1', '11_1', '02_2', '10_1'],
        'K559': ['08_2', '12_1', '02_1', '08_1', '02_2', '10_1'],
        'K561': ['08_2', '01_1', '02_1', '08_1', '09_1', '02_2', '10_1', '05_1', '05_2', '06_1'],
        'K198': ['052', '061', '112', '021', '051', '111', '081'],
        'K704': ['12_1', '02_1', '69_1', '11_1', '65_2', '11_3', '65_1', '11_2'],
        'K702': ['03_1', '11_1', '05_1', '04_1', '05_2', '11_2'],
        'K703': ['08_2', '02_1', '08_1', '03_1', '09_1', '05_1', '05_2'],
        # 'K159': ['021', '051', '713', '111', '081'],
        'K159': ['021', '051', '111', '081'],
        'K182': ['621', '682', '681', '622', '121'],
        'K183': ['082', '021', '051', '022', '111', '081'],
        'K128': ['131', '161', '061', '112', '162', '062'],
        # 'K139': ['101', '082', '021', '022', '121', '081'],
        # 'K104': ['082', '021', '051', '022', '111', '081'],
        'K101': ['621', '082', '051', '711', '622', '121', '081'],
        'K206': ['091', '052', '101', '082', '061', '021', '051', '022', '092', '081'],
        'K074': ['052', '051'],
        'K414': ['091', '122', '082', '051', '011', '012', '121', '092', '081'],
        'K415': ['061', '021', '081', '041'],
        'K250': ['091', '101', '102', '021', '022', '092', '081']
        }

# Base path for the sensor data
base_path = 'data/Hauge'

Identify missing timestamps

In [ ]:
# Generate a complete DateTimeIndex from 2018-01-01 00:00:00 to 2019-12-31 23:55:00 with 5-minute intervals
complete_index = pd.date_range(start='2018-01-01 00:00:00', end='2019-12-31 23:55:00', freq='5T')

intersections = []
missing_timestamps_dict = {}  # Initialize a dictionary to store missing timestamps for each sensor

for sensor, codes in sensors_dict.items():
    sensor_folder_path = os.path.join(base_path, sensor)
    csv_files = glob.glob(f"{sensor_folder_path}/{sensor}-201[89]-*-*.csv")
    all_data = []
    for file in tqdm(csv_files, desc=f"Processing {sensor}"):
        # Read the CSV file
        df = pd.read_csv(file, delimiter=';', parse_dates=[0], index_col=0)
        
        # Clean the column names of quotes
        df.columns = [col.replace('"', '').replace("'", "") for col in df.columns]
        df = df[~df.index.astype(str).str.contains('totaal')]
    
        df.index.name = None
        df.index = pd.to_datetime(df.index)
        
        # Keep only the filtered columns
        df = df.loc[:, codes]
        
        # Append the DataFrame to the list
        all_data.append(df)

    # Concatenate all DataFrames into a single DataFrame
    final_df = pd.concat(all_data)

    # Sort the DataFrame by the index (date and time)
    final_df.sort_index(inplace=True)
    final_df.columns = [sensor + "-" + col for col in final_df.columns]
    final_df = final_df[final_df.index.year.isin([2018, 2019])]
    
    # Identify missing timestamps
    missing_timestamps = complete_index.difference(final_df.index)
    
    # Store the missing timestamps in the dictionary
    missing_timestamps_dict[sensor] = missing_timestamps

    # Print the first few rows to verify
    # print(final_df.head())
#     intersections.append(final_df)

# hague = pd.concat(intersections, axis=1)
# hague = hague.sort_index(axis=1)
# missing_timestamps_dict

save the missing timestamp

In [ ]:
import json

# Convert datetime objects to strings in missing_timestamps_dict
for sensor, timestamps in missing_timestamps_dict.items():
    missing_timestamps_dict[sensor] = [timestamp.isoformat() for timestamp in timestamps]

# Save missing_timestamps_dict to a JSON file
with open('missing_timestamps.json', 'w') as f:
    json.dump(missing_timestamps_dict, f, indent=4)

# The file missing_timestamps.json now contains the missing timestamps data

Prepare and generate the standardized data file

In [ ]:
intersections = []

for sensor, codes in sensors_dict.items():
    sensor_folder_path = os.path.join(base_path, sensor)
    print(sensor_folder_path)

    # List all CSV files in the sensor folder for the date range 2018-01 to 2019-12
    csv_files = glob.glob(f"{sensor_folder_path}/{sensor}-201[89]-*-*.csv")
    all_data = []
    for file in csv_files:
        # Read the CSV file
        df = pd.read_csv(file, delimiter=';', parse_dates=[0], index_col=0)
        
        # Clean the column names of quotes
        df.columns = [col.replace('"', '').replace("'", "") for col in df.columns]
        df = df[~df.index.astype(str).str.contains('totaal')]
    
        df.index.name = None
        df.index = pd.to_datetime(df.index)
        # print(df.head())
        
        # Keep only the filtered columns
        df = df.loc[:, codes]
        # print(df.head())
        
        # Append the DataFrame to the list
        all_data.append(df)

    # Concatenate all DataFrames into a single DataFrame
    final_df = pd.concat(all_data)

    # Sort the DataFrame by the index (date and time)
    final_df.sort_index(inplace=True)
    final_df.columns = [sensor + "-" + col for col in final_df.columns]
    final_df = final_df[final_df.index.year.isin([2018, 2019])]
    

    # Print the first few rows to verify
    # print(final_df.head())
    non_unique_indices = final_df.index.duplicated(keep=False)
    non_unique_rows = final_df[non_unique_indices]

    if not non_unique_rows.empty:
        print(non_unique_rows)
    intersections.append(final_df)
# hague = pd.concat(intersections, axis=1)
# hague = hague.sort_index(axis=1)

# hague.head()

In [ ]:
hague = pd.concat(intersections, axis=1)
hague = hague.sort_index(axis=1)

hague.head()

In [ ]:
hague.to_csv("data/Hauge/hague.csv")

### Try to impute the missing values

Only select the columns with missing values < 34%

In [ ]:
nan_percentage = hague.isna().mean() * 100

In [ ]:
print(nan_percentage.sort_values(ascending=False)[:25])

In [ ]:
columns_below_34_nan = nan_percentage[nan_percentage < 34].index.tolist()

# Selecting the columns in 'hague' DataFrame where the NaN percentage is less than 34%
filtered_hague = hague[columns_below_34_nan]

Imputation based on https://www.geo.fu-berlin.de/en/v/soga-py/Advanced-statistics/time-series-analysis/Dealing-with-missing-values/Imputing-missing-values/index.html

In [ ]:
filtered_hague.head()

In [ ]:
temp_NA_inter = filtered_hague.interpolate(method="time").copy()

In [ ]:
temp_NA_inter.head()

In [ ]:
temp_NA_bfill = temp_NA_inter.fillna(method="bfill")
temp_NA_bfill.head()

In [ ]:
temp_NA_bfill.to_hdf('data/Hauge/hague_filled.h5', key='df', mode='w')

# Create new adjacency matrix Hague

In [ ]:
distances_df = pd.read_csv('data/Hauge/distances.csv')
df = pd.read_hdf('data/Hauge/hague_filled.h5')
df.head()

In [ ]:
sensor_ids = df.columns.astype(str).tolist()

# Efficiently convert sensor IDs to index for quick lookup (now all as strings)
sensor_id_to_index = {str(sensor_id): index for index, sensor_id in enumerate(sensor_ids)}

# Initialize an empty distance matrix
num_sensors = len(sensor_ids)
dist_mx_array = np.full((num_sensors, num_sensors), np.inf)

# Make sure 'from' and 'to' in distances_df are also strings
distances_df['from'] = distances_df['from'].astype(str)
distances_df['to'] = distances_df['to'].astype(str)

In [ ]:
# Populate the distance matrix with the data from distances_df
for _, row in distances_df.iterrows():
    from_sensor = row['from']
    to_sensor = row['to']
    if from_sensor in sensor_id_to_index and to_sensor in sensor_id_to_index:
        i = sensor_id_to_index[from_sensor]
        j = sensor_id_to_index[to_sensor]
        dist_mx_array[i, j] = row['cost']

# Replace the diagonal with zeros since the distance from a sensor to itself is zero
np.fill_diagonal(dist_mx_array, 0)


In [ ]:
def get_adjacency_matrix(dist_mx_array, normalized_k=0.1):
    # Calculate the standard deviation as theta for normalization
    distances = dist_mx_array[~np.isinf(dist_mx_array)].flatten()
    std = distances.std()
    adj_mx_array = np.exp(-np.square(dist_mx_array / std))
    
    # Set entries below a threshold to zero for sparsity
    adj_mx_array[adj_mx_array < normalized_k] = 0
    return adj_mx_array

In [ ]:
dist_mx_array

In [ ]:
adj_mx_array = get_adjacency_matrix(dist_mx_array).astype(np.float32)

# Construct the adj_mx list as specified with the updated adj_mx_array
adj_mx = [sensor_ids, sensor_id_to_index, adj_mx_array]

In [ ]:
adj_mx[2][0]

In [ ]:
# Construct the file path
directory = 'data/Hauge'
file_name = 'adj_mx.pkl'
file_path = os.path.join(directory, file_name)

# Ensure the directory exists
os.makedirs(directory, exist_ok=True)

# Save the adj_mx to a .pkl file
with open(file_path, 'wb') as f:
    pickle.dump(adj_mx, f)

# Create a smaller size of the original data

In [ ]:
import pandas as pd
df = pd.read_hdf('data/Hauge/hague_filled.h5')

In [ ]:
total_rows = len(df)

# Calculate indices for 12.5th and 87.5th percentiles
start_index = int(total_rows * 0.125)
end_index = int(total_rows * 0.875)

# Select the middle 75% of the rows
middle_75_df = df.iloc[start_index:end_index]
middle_75_df.head()

In [ ]:
middle_75_df.to_hdf('data/Hauge/hague_filled_75.h5', key='df', mode='w')

# Validate

In [ ]:
import pickle

# file_path = 'data/METRLA/adj_mx_new.pkl'
file_path = 'data/Hauge/adj_mx.pkl'

with open(file_path, 'rb') as f:
    adj_mx = pickle.load(f, encoding='latin1')

print(len(adj_mx))
print(adj_mx[0])
print(len(adj_mx[0]))
print(adj_mx[1])

In [ ]:
adj_mx[2]

In [ ]:
adj_mx[2].T

# Transform Hauge to a compressed sensor datasets

In [ ]:
file_path = 'data/Hauge/Hague_comp_sensor_ref.xlsx'
df1 = pd.read_excel(file_path)
df2 = pd.read_csv('data/Hauge/hague.csv', index_col=0)

In [ ]:
def process_dataframes(df1, df2):
    # New dataframe to hold results
    df3 = pd.DataFrame()
    
    # Iterate over rows in df1
    for idx, row in df1.iterrows():
        intersection = row['Intersection']
        original_columns = str(row['Original']).split(',')

        # Create full column names in df2 and sum them
        if len(original_columns) > 1:
            new_column_name = intersection + '-' + row['New']
            # Create a list of full names to search in df2
            df2_column_names = [intersection + '-' + orig for orig in original_columns]
            # Average the columns in df2 that are in our list if they exist
            df3[new_column_name] = df2[df2_column_names].mean(axis=1)
        else:
            # Just copy the column from df2
            df2_column_name = intersection + '-' + original_columns[0]
            df3[df2_column_name] = df2[df2_column_name] if df2_column_name in df2.columns else pd.Series()
    
    return df3

In [ ]:
df3 = process_dataframes(df1, df2)

In [ ]:
df3.columns

In [ ]:
df3.head()

In [ ]:
df3.columns

In [ ]:
df3.to_csv("data/Hauge/hague_comp.csv")

In [ ]:
nan_percentage = df3.isna().mean() * 100
print(nan_percentage.sort_values(ascending=False)[:25])

In [ ]:
columns_below_34_nan = nan_percentage[nan_percentage < 30].index.tolist()

# Selecting the columns in 'hague' DataFrame where the NaN percentage is less than 34%
filtered_hague = df3[columns_below_34_nan]
filtered_hague.index = pd.to_datetime(filtered_hague.index)

In [ ]:
temp_NA_inter = filtered_hague.interpolate(method="time").copy()
temp_NA_bfill = temp_NA_inter.fillna(method="bfill")
temp_NA_bfill.to_hdf('data/Hauge/hague_comp_filled.h5', key='df', mode='w')

# New adj based on combined sensors

In [28]:
import pandas as pd

In [29]:
df1 = pd.read_csv('data/HAUGE/distances.csv')
df2 = pd.read_hdf('data/Hauge/hague_comp_filled.h5')
df3 = pd.read_excel('data/Hauge/Hague_comp_sensor_ref.xlsx')
df4 = pd.read_csv('data/Hauge/location_std.csv')

In [30]:
def transform_id(sensor_id, trajectory, df3):
    # Split the sensor_id by '-'
    parts = sensor_id.split('-')
    if len(parts) != 2:
        return None  # Return None if sensor_id does not match expected format
    
    # Find matching row in df3 with the same trajectory
    mask = (df3['Intersection'] == parts[0]) & (df3['Representation'] == parts[1]) & (df3['Trajectory'] == trajectory)
    if mask.any():
        # If a matching row is found, replace with the 'New' value
        return parts[0] + '-' + df3.loc[mask, 'New'].values[0]
    return None

In [31]:
def apply_transform_or_keep_original(row, df3):
    trajectories = row['trajectory'].split('+')
    new_ids = []

    for trajectory in trajectories:
        new_id = transform_id(row['sensor_id'], trajectory, df3)
        if new_id is not None:
            new_ids.append(new_id)

    # If no new ID was found for either trajectory, keep the original sensor_id
    if not new_ids:
        new_ids.append(row['sensor_id'])

    # Return a list of dictionaries for each new_id to expand into new rows
    return [{'sensor_id': new_id, 'latitude': row['latitude'], 'longitude': row['longitude'], 'trajectory': trajectory} for new_id, trajectory in zip(new_ids, trajectories)]

# Apply the transformation to generate new rows
expanded_rows = df4.apply(lambda row: apply_transform_or_keep_original(row, df3), axis=1)

# Flatten the list of lists into a single DataFrame
df4_expanded = pd.DataFrame([item for sublist in expanded_rows for item in sublist])

# Keep only the sensor IDs that are present in the columns of df2
df2_columns = set(df2.columns)
df4_filtered = df4_expanded[df4_expanded['sensor_id'].isin(df2_columns)]
df4_filtered

,sensor_id,latitude,longitude,trajectory
1,K502-S1,52.051972,4.229278,T1N
2,K502-051,52.051750,4.228972,T1N
3,K502-111,52.052000,4.229806,T1N
11,K504-N1,52.042250,4.240278,T1N
14,K504-11_1,52.042500,4.239972,T1N
...,...,...,...,...
164,K415-021,52.071609,4.341984,T2N
165,K415-081,52.071553,4.341502,T2S
167,K250-N1,52.067711,4.353475,T2N
169,K250-N2,52.067414,4.353624,T2N


In [32]:
df4_filtered.to_csv('data/Hauge/location_std_comp.csv', index=False)

In [ ]:
df1['new_from'] = df1.apply(lambda row: transform_id(row['from'], row['trajectory'], df3), axis=1)
df1['new_to'] = df1.apply(lambda row: transform_id(row['to'], row['trajectory'], df3), axis=1)

In [ ]:
df1.head()

In [ ]:
df2_columns = set(df2.columns)
df4 = df1[(df1['new_from'].isin(df2_columns)) & (df1['new_to'].isin(df2_columns))]
df4.head()

In [ ]:
df4['from'] = df4['new_from']
df4['to'] = df4['new_to']
df4 = df4.drop(['new_from', 'new_to'], axis=1)
df4.head()

In [ ]:
df4.to_csv('data/Hauge/distances_comp.csv', index=False)

## Generate the adjacency matrix for the combined sensors

In [ ]:
distances_df = pd.read_csv('data/Hauge/distances_comp.csv')
df = pd.read_hdf('data/Hauge/hague_comp_filled.h5')

sensor_ids = df.columns.astype(str).tolist()

# Efficiently convert sensor IDs to index for quick lookup (now all as strings)
sensor_id_to_index = {str(sensor_id): index for index, sensor_id in enumerate(sensor_ids)}

# Initialize an empty distance matrix
num_sensors = len(sensor_ids)
dist_mx_array = np.full((num_sensors, num_sensors), np.inf)

# Make sure 'from' and 'to' in distances_df are also strings
distances_df['from'] = distances_df['from'].astype(str)
distances_df['to'] = distances_df['to'].astype(str)

# Populate the distance matrix with the data from distances_df
for _, row in distances_df.iterrows():
    from_sensor = row['from']
    to_sensor = row['to']
    if from_sensor in sensor_id_to_index and to_sensor in sensor_id_to_index:
        i = sensor_id_to_index[from_sensor]
        j = sensor_id_to_index[to_sensor]
        dist_mx_array[i, j] = row['cost']

# Replace the diagonal with zeros since the distance from a sensor to itself is zero
np.fill_diagonal(dist_mx_array, 0)

In [ ]:
def get_adjacency_matrix(dist_mx_array, normalized_k=0.1):
    # Calculate the standard deviation as theta for normalization
    distances = dist_mx_array[~np.isinf(dist_mx_array)].flatten()
    std = distances.std()
    adj_mx_array = np.exp(-np.square(dist_mx_array / std))
    
    # Set entries below a threshold to zero for sparsity
    adj_mx_array[adj_mx_array < normalized_k] = 0
    return adj_mx_array

In [ ]:
adj_mx_array = get_adjacency_matrix(dist_mx_array).astype(np.float32)

# Construct the adj_mx list as specified with the updated adj_mx_array
adj_mx = [sensor_ids, sensor_id_to_index, adj_mx_array]
adj_mx[2][0]

In [ ]:
# Construct the file path
directory = 'data/Hauge'
file_name = 'adj_mx_comp1.pkl'
file_path = os.path.join(directory, file_name)

# Ensure the directory exists
os.makedirs(directory, exist_ok=True)

# Save the adj_mx to a .pkl file
with open(file_path, 'wb') as f:
    pickle.dump(adj_mx, f)

## Generate the adjacency matrix for the combined sensors with direction applied to only count in foward direction that are connected by lanes

In [ ]:
distances_df = pd.read_csv('data/Hauge/distances_comp_directed.csv')
df = pd.read_hdf('data/Hauge/hague_comp_filled.h5')

sensor_ids = df.columns.astype(str).tolist()

# Efficiently convert sensor IDs to index for quick lookup (now all as strings)
sensor_id_to_index = {str(sensor_id): index for index, sensor_id in enumerate(sensor_ids)}

# Initialize an empty distance matrix
num_sensors = len(sensor_ids)
dist_mx_array = np.full((num_sensors, num_sensors), np.inf)

# Make sure 'from' and 'to' in distances_df are also strings
distances_df['from'] = distances_df['from'].astype(str)
distances_df['to'] = distances_df['to'].astype(str)

# Populate the distance matrix with the data from distances_df
for _, row in distances_df.iterrows():
    from_sensor = row['from']
    to_sensor = row['to']
    if from_sensor in sensor_id_to_index and to_sensor in sensor_id_to_index:
        i = sensor_id_to_index[from_sensor]
        j = sensor_id_to_index[to_sensor]
        dist_mx_array[i, j] = row['cost']

# Replace the diagonal with zeros since the distance from a sensor to itself is zero
np.fill_diagonal(dist_mx_array, 0)

In [ ]:
def get_adjacency_matrix(dist_mx_array, normalized_k=0.1):
    # Calculate the standard deviation as theta for normalization
    distances = dist_mx_array[~np.isinf(dist_mx_array)].flatten()
    std = distances.std()
    adj_mx_array = np.exp(-np.square(dist_mx_array / std))
    
    # Set entries below a threshold to zero for sparsity
    adj_mx_array[adj_mx_array < normalized_k] = 0
    return adj_mx_array

In [ ]:
adj_mx_array = get_adjacency_matrix(dist_mx_array).astype(np.float32)

# Construct the adj_mx list as specified with the updated adj_mx_array
adj_mx = [sensor_ids, sensor_id_to_index, adj_mx_array]
adj_mx[2][0]

In [ ]:
# Construct the file path
directory = 'data/Hauge'
file_name = 'adj_mx_comp2.pkl'
file_path = os.path.join(directory, file_name)

# Ensure the directory exists
os.makedirs(directory, exist_ok=True)

# Save the adj_mx to a .pkl file
with open(file_path, 'wb') as f:
    pickle.dump(adj_mx, f)

# Save a copy of adjacency matrix to csv file for inspection

In [ ]:
import numpy as np
import pickle

METR-LA

In [ ]:
file_path = 'data/METRLA/adj_mx.pkl'
with open(file_path, 'rb') as f:
    adj_mx = pickle.load(f, encoding='latin1')
np.savetxt("data/METRLA/adj_mx.csv", adj_mx[2], delimiter=",", fmt='%.4f')

In [ ]:
file_path = 'data/METRLA/adj_mx_new1.pkl'
with open(file_path, 'rb') as f:
    adj_mx = pickle.load(f, encoding='latin1')
np.savetxt("data/METRLA/adj_mx_new1.csv", adj_mx[2], delimiter=",", fmt='%.4f')

PeMS-BAY

In [ ]:
file_path = 'data/PEMSBAY/adj_mx_bay.pkl'
with open(file_path, 'rb') as f:
    adj_mx = pickle.load(f, encoding='latin1')
np.savetxt("data/PEMSBAY/adj_mx_bay.csv", adj_mx[2], delimiter=",", fmt='%.4f')

In [ ]:
file_path = 'data/PEMSBAY/adj_mx_new1.pkl'
with open(file_path, 'rb') as f:
    adj_mx = pickle.load(f, encoding='latin1')
np.savetxt("data/PEMSBAY/adj_mx_new1.csv", adj_mx[2], delimiter=",", fmt='%.4f')

Hauge

In [ ]:
file_path = 'data/Hauge/adj_mx1.pkl'
with open(file_path, 'rb') as f:
    adj_mx = pickle.load(f, encoding='latin1')
np.savetxt("data/Hauge/adj_mx1.csv", adj_mx[2], delimiter=",", fmt='%.4f')

In [ ]:
file_path = 'data/Hauge/adj_mx_comp1.pkl'
with open(file_path, 'rb') as f:
    adj_mx = pickle.load(f, encoding='latin1')
np.savetxt("data/Hauge/adj_mx_comp1.csv", adj_mx[2], delimiter=",", fmt='%.4f')

In [ ]:
file_path = 'data/Hauge/adj_mx_comp2.pkl'
with open(file_path, 'rb') as f:
    adj_mx = pickle.load(f, encoding='latin1')
np.savetxt("data/Hauge/adj_mx_comp2.csv", adj_mx[2], delimiter=",", fmt='%.4f')

# Small size dataset Hague for cost model (memory saver)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_hdf('data/Hauge/hague_comp_filled.h5')

Middle 20

In [ ]:
# Calculate indices for the middle 20%
start_idx = int(len(df) * 0.40)
end_idx = int(len(df) * 0.60)

# Select the middle 20% chunk
df_middle_20_percent = df.iloc[start_idx:end_idx]

# Convert index to datetime
df_middle_20_percent.index = pd.to_datetime(df_middle_20_percent.index, unit='ns')

# Save to HDF5 file
df_middle_20_percent.to_hdf('data/Hauge/hague_comp_filled_20_2.h5', key='df', mode='w')

First 20

In [ ]:
# Calculate indices for the middle 20%
start_idx = int(len(df) * 0.0)
end_idx = int(len(df) * 0.2)

# Select the middle 20% chunk
df_first_20_percent = df.iloc[start_idx:end_idx]

# Convert index to datetime
df_first_20_percent.index = pd.to_datetime(df_first_20_percent.index, unit='ns')

# Save to HDF5 file
df_first_20_percent.to_hdf('data/Hauge/hague_comp_filled_20_3.h5', key='df', mode='w')

# Locations

In [ ]:
import pandas as pd
import folium

# Load the CSV data
csv_file_path = 'data/METRLA/graph_sensor_locations.csv'
data = pd.read_csv(csv_file_path)

# Create a folium map centered around the mean latitude and longitude
center_lat = data['latitude'].mean()
center_lon = data['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='OpenStreetMap')

# Add sensor locations to the map
for idx, row in data.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        popup=f'Sensor ID: {row["sensor_id"]}'
    ).add_to(m)

# Save the map to an HTML file
m.save('sensor_locations_map_METRLA.html')

m

In [ ]:
import pandas as pd
import folium

# Load the CSV data
csv_file_path = 'data/PEMSBAY/graph_sensor_locations_bay.csv'
data = pd.read_csv(csv_file_path)

# Create a folium map centered around the mean latitude and longitude
center_lat = data['latitude'].mean()
center_lon = data['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='OpenStreetMap')

# Add sensor locations to the map
for idx, row in data.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        popup=f'Sensor ID: {row["sensor_id"]}'
    ).add_to(m)

# Save the map to an HTML file
m.save('sensor_locations_map_bay.html')

m

In [ ]:
import pandas as pd
import folium

# Load the CSV data
csv_file_path = 'data/Hauge/location_std.csv'
data = pd.read_csv(csv_file_path)

# Extract the first trajectory if multiple are concatenated
data['trajectory'] = data['trajectory'].apply(lambda x: x.split('+')[0])

# Predefined colors for the trajectories
color_mapping = {
    'T1N': 'red',
    'T1S': 'orange',
    'T2N': 'blue',
    'T2S': 'purple'
}

# Create a folium map centered around the mean latitude and longitude
center_lat = data['latitude'].mean()
center_lon = data['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='OpenStreetMap')

# Add sensor locations to the map with specific colors for each trajectory
for idx, row in data.iterrows():
    trajectory = row['trajectory']
    color = color_mapping.get(trajectory, 'gray')  # Default to gray if trajectory is not in color_mapping
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f'Sensor ID: {row["sensor_id"]}, Trajectory: {row["trajectory"]}'
    ).add_to(m)

# Save the map to an HTML file
m.save('sensor_locations_map_Hauge.html')

m